In [ ]:
https://github.com/muneer-kayali/autodubbing

In [ ]:
!ffmpeg -y -i video_input.mp4 -q:a 0 audio_input.mp3

ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame -

In [1]:
import stable_whisper
model = stable_whisper.load_model('large')

In [22]:
import os
video = "15"
video2 = "16"
audio = os.path.join("data", "processed", f"video_{video}", "extracted_audio.wav")
audio2 = os.path.join("data", "processed", f"video_{video2}", "extracted_audio.wav")
result = model.transcribe(audio, vad=True, min_word_dur=0.3, vad_threshold=0.45, nonspeech_error=0.25)
result2 = model.transcribe(audio2, vad=True, min_word_dur=0.3, vad_threshold=0.45, nonspeech_error=0.25)

c:\Users\willi\anaconda3\envs\AutoDubbing\Lib\site-packages\stable_whisper\whisper_word_level\original_whisper.py:249: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Transcribe:   0%|          | 0/98.01 [01:12<?, ?sec/s]

Detected language: english


Transcribe: 100%|██████████| 98.01/98.01 [08:53<00:00,  5.44s/sec]
Transcribe:   0%|          | 0/81.15 [01:02<?, ?sec/s]

Detected language: english


Transcribe: 100%|██████████| 81.15/81.15 [02:51<00:00,  2.12s/sec]


In [25]:
all_segments = []
for segment in result2.segments:
    all_segments.append({"start": segment.start, "end": segment.end, "text": segment.text})

In [26]:
for i, segment in enumerate(all_segments):
    print(f"Segment {i + 1}: '{segment['text']}', Start: '{segment['start']}, End: '{segment['end']}")

Segment 1: ' Oh, hi.', Start: '0.2, End: '0.78
Segment 2: ' You know what?', Start: '0.96, End: '1.84
Segment 3: ' Um,', Start: '2.08, End: '2.22
Segment 4: ' before I get in the box,', Start: '2.24, End: '3.8
Segment 5: ' could I just run to the restroom', Start: '3.94, End: '5.0
Segment 6: ' and make sure my hair is perfect?', Start: '5.0, End: '6.22
Segment 7: ' Fine.', Start: '6.68, End: '6.92
Segment 8: ' Can you be speedy about it?', Start: '7.16, End: '8.12
Segment 9: ' Mm-hmm.', Start: '8.2, End: '8.42
Segment 10: ' Down this way?', Start: '10.16, End: '10.86
Segment 11: ' It's just down the hallway.', Start: '10.96, End: '11.88
Segment 12: ' Thank you.', Start: '12.3, End: '12.72
Segment 13: ' On the right.', Start: '13.2, End: '13.62
Segment 14: ' Guess you really had to go to the bathroom.', Start: '18.1, End: '19.68
Segment 15: ' Get that Barbie!', Start: '20.74, End: '22.176
Segment 16: ' Don't you dare push that button.', Start: '25.02, End: '26.18
Segment 17: ' Let me pu

In [33]:
import subprocess

api_key = "ENTER YOUR OPENAI API KEY HERE"

# Set the environment variable using PowerShell
command = ["powershell", "-Command", f"setx OPENAI_API_KEY \"{api_key}\""]
result1 = subprocess.run(command, capture_output=True, text=True)

# Check if the command succeeded
if result1.returncode == 0:
    print(f"Environment variable OPENAI_API_KEY set successfully:\n{result1.stdout}")
else:
    print(f"Failed to set environment variable:\n{result1.stderr}")

Environment variable OPENAI_API_KEY set successfully:

SUCCESS: Specified value was saved.



In [35]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()

def group_and_merge_segments_by_speaker(segments):
    """
    Groups segments by speaker and merges adjacent segments spoken by the same speaker.

    Args:
        segments (list): List of segments, each with 'text', 'start', and 'end' keys.

    Returns:
        list: Merged segments with speaker information.
    """
    merged_segments = []  # List to store the final merged segments

    # Construct the prompt with all segments
    prompt_intro = (
        "You are an expert at analyzing conversational text to determine speakers in a dialogue. "
        "The text segments provided are from a conversation involving two speakers or more speakers: 'Speaker 1', 'Speaker 2', ..., 'Speaker n'. "
        "Your task is to classify each segment into one of these speakers based on their style, content, or clues. "
        "If a segment clearly belongs to one speaker, respond with 'Speaker i' for i in all speakers. "
        "If the speaker cannot be confidently determined, force yourself to make a decision. Furthermore, you should group the segments if two or more adjacent segments are spoken by the same speaker, however, this should only be done if the sentences are spoken right away (if there is silence of 2 seconds or more then dont group), Also, the segments should not be long, so if regrouping gives you a long segment of multiple sentences (2-3), then this should not be done. Here are the segments:\n\n"
    )

    # Append all segments to the prompt
    for i, segment in enumerate(segments):
        prompt_intro += f"Segment {i + 1}: '{segment['text']}', Start: '{segment['start']}, End: '{segment['end']}\n"

    prompt_intro += (
        "\nPlease provide your output after classifying and regrouping in the following format:\n"
        "Speaker: speaker_num, Text: text, Start: start, End: end\n"
        "Speaker: speaker_num, Text: text, Start: start, End: end\n"
        "Speaker: speaker_num, Text: text, Start: start, End: end\n"
        "Avoid formatting the text with bold or italics. Use the exact format as shown above.\n"
    )

    # Call the OpenAI API with the constructed prompt
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_intro}
        ]
    )

    # Extract the model's response
    response = completion.choices[0].message.content.strip()
    print("API Response:\n", response)
    
    regrouped_segments = []

    # Split response into lines
    response_lines = response.splitlines()

    for line in response_lines:
        line = line.strip()
        
        # Skip empty lines or irrelevant content
        if not line or not line.startswith("Speaker:"):
            continue

        try:
            # Parse the line into components
            speaker = line.split("Speaker:")[1].split(", Text:")[0].strip()
            text = line.split("Text:")[1].split(", Start:")[0].strip()
            start = float(line.split("Start:")[1].split(", End:")[0].strip())
            end = float(line.split("End:")[1].strip())

            # Add parsed segment to the list
            regrouped_segments.append({
                "speaker": speaker,
                "text": text,
                "start": start,
                "end": end,
            })
        except (IndexError, ValueError) as e:
            print(f"Error parsing line: {line}, Error: {e}")

    return regrouped_segments

regrouped_segments = group_and_merge_segments_by_speaker(all_segments)


API Response:
 Speaker: Speaker 1, Text: Let's take a look at the weather forecast now., Start: 0.192, End: 2.62
Speaker: Speaker 1, Text: I'm delighted to say we've got a new member of our weather team tonight. Let me hand over to him now., Start: 2.82, End: 8.096
Speaker: Speaker 2, Text: Well, it's an unsettled picture as we head towards the end of the week., Start: 8.224, End: 13.888
Speaker: Speaker 2, Text: This afternoon it'll be cold, wet and windy across most of Scotland. We're under the influence of low pressure and this weather front pushing northwards is bringing cloud and outbreaks of rain., Start: 14.592, End: 27.808
Speaker: Speaker 2, Text: The rain, of course, will be heaviest over the borders and around Edinburgh, where it could lead to difficult conditions on the roads., Start: 27.968, End: 35.584
Speaker: Speaker 2, Text: In the west, rain will be lighter and patchier. There will maybe a few drier interludes over Dumfries House in Ayrshire., Start: 36.384, End: 43.2

In [38]:
from pysrt import SubRipFile
from pydub import AudioSegment
import os
import tempfile

audio = AudioSegment.from_mp3("audio_input.mp3")

# Function to slice audio based on .srt segments
def slice_audio_by_srt(audio, segments):
    output_dir = "tempfiles"
    os.makedirs(output_dir, exist_ok=True)

    audio_segments = []  # List to store audio segments

    for i, segment in enumerate(segments):
        # Convert start and end times to milliseconds
        start_time = int(segment["start"] * 1000)  # Convert seconds to milliseconds
        end_time = int(segment["end"] * 1000)  # Convert seconds to milliseconds

        # Slice the audio based on timestamps
        audio_slice = audio[start_time:end_time]
        audio_segments.append(audio_slice)

        # Create a file path for the sliced audio
        segment_path = os.path.join(output_dir, f"segment{i}.wav")

        # Save each segment as a separate file
        audio_slice.export(segment_path, format="wav")
        print(f"Saved: {segment_path}")

    return audio_segments

sliced_segments = slice_audio_by_srt(audio, regrouped_segments)

Saved: tempfiles\segment0.wav
Saved: tempfiles\segment1.wav
Saved: tempfiles\segment2.wav
Saved: tempfiles\segment3.wav
Saved: tempfiles\segment4.wav
Saved: tempfiles\segment5.wav
Saved: tempfiles\segment6.wav
Saved: tempfiles\segment7.wav
Saved: tempfiles\segment8.wav
Saved: tempfiles\segment9.wav
Saved: tempfiles\segment10.wav


In [39]:
from google.cloud import texttospeech
def synthesize_text(input, ssml=False):
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    
    client = texttospeech.TextToSpeechClient()
    if not ssml:
        input_text = texttospeech.SynthesisInput(text=input)
    else:
        input_text = texttospeech.SynthesisInput(ssml=input)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code="da-DK",
        name="da-DK-Neural2-D",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    # The response's audio_content is binary.
    return response.audio_content
    # with open("output.mp3", "wb") as out:
    #     out.write(response.audio_content)
    #     print('Audio content written to file "output.mp3"')

In [40]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "SET PATH TO GOOGLE CLOUD JSON FILE HERE..."

In [41]:
import deepl

auth_key = "ENTER YOUR DEEPL KEY HERE..."  # Replace with your key
translator = deepl.Translator(auth_key)

def translate(text):
    result = translator.translate_text(text, target_lang="DA")
    return result.text


In [42]:
translated_segments = []
for segment in regrouped_segments:
    translated_text = translate(segment["text"])
    translated_segments.append({
        "start": segment['start'],
        "end": segment['end'],
        "target_duration": segment['end'] - segment['start'],
        "text": translated_text
    })
    

In [43]:
translated_audio_files = []
for segment in translated_segments:
    translated_audio_files.append(synthesize_text(segment['text']))

In [47]:
from pydub import AudioSegment
from io import BytesIO
durations = []
for segment in translated_audio_files:
    audio = AudioSegment.from_file(BytesIO(segment), format="mp3")
    duration = len(audio)
    durations.append(duration)
durations = [d / 1000 for d in durations]


In [48]:
for i, segment in enumerate(translated_segments):
    segment['source_duration'] = durations[i]

In [49]:
for i, audio_file in enumerate(translated_audio_files):
    with open(os.path.join("tempfiles", f"translated_segment{i}.wav"), "wb") as f:
        f.write(audio_file)

In [50]:
def split_audio_by_speaker(grouped_segments, original_audio_path, output_dir):
    """
    Splits audio into separate tracks for each speaker and saves them as separate files.

    Args:
        grouped_segments (list): List of dictionaries containing speaker, text, start, and end timestamps.
        original_audio_path (str): Path to the original audio file.
        output_dir (str): Directory to save the split audio files.

    Returns:
        None
    """
    # Load the original audio
    audio = AudioSegment.from_file(original_audio_path)

    # Dictionary to store audio for each speaker
    speaker_audio = {}

    # Iterate through grouped segments and append audio to the respective speaker
    for entry in grouped_segments:
        speaker = entry["speaker"]
        start_time = int(entry["start"] * 1000)  # Convert to milliseconds
        end_time = int(entry["end"] * 1000)  # Convert to milliseconds

        # Initialize silent audio for the speaker if not already in dictionary
        if speaker not in speaker_audio:
            speaker_audio[speaker] = AudioSegment.silent(duration=0)

        # Slice the audio and append to the corresponding speaker
        speaker_audio[speaker] += audio[start_time:end_time]

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Save the audio files for each speaker
    for speaker, speaker_track in speaker_audio.items():
        output_path = os.path.join(output_dir, f"{speaker.replace(' ', '_').lower()}.wav")
        speaker_track.export(output_path, format="wav")
        print(f"Saved {speaker} audio to: {output_path}")
        
original_audio_path = "audio_input.mp3"  # Path to the original audio file
output_dir = "speaker_audio"  # Directory to save the output files
split_audio_by_speaker(regrouped_segments, original_audio_path, output_dir)

Saved Speaker 1 audio to: speaker_audio\speaker_1.wav
Saved Speaker 2 audio to: speaker_audio\speaker_2.wav


In [51]:
import os
import subprocess
from pydub import AudioSegment

# Function to calculate audio duration in seconds
def get_audio_duration(file_path):
    audio = AudioSegment.from_file(file_path)
    return len(audio) / 1000.0  # Duration in seconds

# Function to determine length adjustment factor
def calculate_length_adjust(source_duration, reference_duration):
    ratio = reference_duration / source_duration
    # Apply bounds
    if ratio > 1.25:
        return 1.25
    elif ratio < 0.75:
        return 0.75
    return ratio

# Function to run SEED-VC for voice conversion
def run_seed_vc(source, target, output_dir, length_adjust):
    os.makedirs(output_dir, exist_ok=True)

    command = [
        ".venv\\Scripts\\python", "seed-vc-main/inference.py",
        "--source", source,
        "--target", target,
        "--output", output_dir,
        "--diffusion-steps", "25",
        "--length-adjust", str(length_adjust),
        "--inference-cfg-rate", "0.7"
    ]
    try:
        subprocess.run(command, check=True, capture_output=True, text=True, encoding="utf-8")
        print(f"Voice conversion completed. Output directory: {output_dir}")
    except subprocess.CalledProcessError as e:
        print(f"Error during voice conversion for {source} and {target}:")
        print(e.stderr)
        raise e

# Paths and directories
temp_dir = os.path.abspath("tempfiles")  # Use absolute path for temp files
output_base_dir = os.path.abspath("vc_segments")  # Base directory for SEED-VC outputs
speaker_audio_dir = os.path.abspath("speaker_audio")  # Directory for speaker audio files

# Dynamically map speaker audio files
speaker_to_file = {}

# List all files in the speaker_audio directory
for file_name in os.listdir(speaker_audio_dir):
    if file_name.endswith(".wav"):  # Ensure it's a WAV file
        # Extract the speaker name from the file name
        speaker_name = file_name.replace(".wav", "").replace("_", " ").title()
        # Map the speaker to the file path
        speaker_to_file[speaker_name] = os.path.join(speaker_audio_dir, file_name)

# Iterate over all segments
converted_segments = []  # To store paths of converted audio segments
for i, segment in enumerate(regrouped_segments):
    speaker = segment["speaker"]
    segment = segment["text"]

    # Files for this segment
    source_file = os.path.join(temp_dir, f"translated_segment{i}.wav")
    reference_length_file = os.path.join(temp_dir, f"segment{i}.wav")  # Used to calculate length_adjust
    target_speaker_file = speaker_to_file[speaker]  # Speaker-specific reference file for conversion
    output_dir = os.path.join(output_base_dir, f"segment_{i}")  # Unique directory for each segment

    # Check if files exist
    if not os.path.exists(source_file) or not os.path.exists(reference_length_file) or not os.path.exists(target_speaker_file):
        print(f"Skipping segment {i}: Missing file(s).")
        continue

    # Calculate durations
    source_duration = get_audio_duration(source_file)
    reference_duration = get_audio_duration(reference_length_file)

    # Determine length adjustment
    length_adjust = calculate_length_adjust(source_duration, reference_duration)

    # Run voice conversion and get the output file
    converted_file = run_seed_vc(source_file, target_speaker_file, output_dir, length_adjust)
    converted_segments.append(converted_file)

# Summary
print(f"Voice conversion completed for {len(converted_segments)} segments.")

Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_0
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_1
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_2
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_3
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_4
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_5
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_6
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_7
Voice conversion completed. Output directory: c:\Users\mimo6\Autodubbing\autodubbing\vc_segments\segment_8
Voice conversion completed. Output di

In [52]:
from pydub import AudioSegment
import os
import glob

# Example data
# `translated_segments` contains start and end times (in seconds) for each segment
# Path to the vc_segments directory
vc_segments_dir = "vc_segments"

# Step 1: Calculate the total duration and create silent audio
max_duration = max(int(data["end"] * 1000) for data in translated_segments)  # In milliseconds
final_audio = AudioSegment.silent(duration=max_duration)

# Step 2: Collect all processed audio files from the vc_segments directory
audio_files = []
for i in range(len(translated_segments)):
    segment_dir = os.path.join(vc_segments_dir, f"segment_{i}")
    # Select any file in the directory (assuming there is only one file per directory)
    segment_files = glob.glob(os.path.join(segment_dir, "*"))  # Matches all files in the directory
    if segment_files:
        audio_files.append(segment_files[0])  # Take the first file found
    else:
        print(f"No file found in {segment_dir}. Skipping...")

# Step 3: Overlay each segment at its corresponding start time
for data, file_path in zip(translated_segments, audio_files):
    start_time = int(data["start"] * 1000)  # Convert start time to milliseconds

    # Load the audio file
    segment = AudioSegment.from_file(file_path)

    # Overlay the segment at the specified start time
    final_audio = final_audio.overlay(segment, position=start_time)

# Step 4: Export the final audio
final_audio.export("output_with_timing.mp3", format="mp3")
print("Final audio saved as output_with_timing.mp3")

Final audio saved as output_with_timing.mp3


In [53]:
import subprocess

# Paths to your video and audio files
video_path = "video_input.mp4"
audio_path = "output_with_timing.mp3"
output_path = "video_output.mp4"



# FFmpeg command to replace the audio
command = [
    "ffmpeg",
    "-y",                   # Overwrite output without asking
    "-i", video_path,       # Input video
    "-i", audio_path,       # Input audio
    "-c:v", "copy",         # Copy video codec (no re-encoding)
    "-c:a", "aac",          # Encode audio with AAC codec
    "-map", "0:v:0",        # Use video from the first input
    "-map", "1:a:0",        # Use audio from the second input
    "-shortest",            # Match the duration of the shorter input
    output_path             # Output file
]

# Run the command
subprocess.run(command, check=True)

print(f"Video with new audio saved at: {output_path}")

Video with new audio saved at: video_output.mp4
